In [ ]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import json

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Load geojson from file
with open('C://Users/USER/Desktop/Master_Irrigation/03_GIS/fields_01.geojson') as f:
    data = json.load(f)
geojsonFc = ee.FeatureCollection(data)

In [ ]:
# Sentinel 1 GRD image collection 
s1_grd = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(geojsonFc).filterDate(ee.Date('2020-04-01'), ee.Date('2020-04-30')) 

In [ ]:
# Map reducer function over imagecollection to get median for multipolygon geometries
s1_grd_rr = ee.FeatureCollection(s1_grd.map(lambda x: x.reduceRegions(collection=geojsonFc,reducer='median', crs='EPSG:4326',scale=10)))

In [ ]:
# Get time series for fields
s1_ts_data = []
s1_ts = s1_grd_rr.toList(10000)
for x in range(0,s1_ts.size().getInfo()):
    s1_ts_data.append(s1_ts.get(x).getInfo())

In [ ]:
# save geojson file for every sentinel 1 grd image a and b seperatly
import geojson
for fc in s1_ts_data:
    name = fc['properties']['system:index']
    with open('C://Users/USER/Desktop/Master_Irrigation/03_GIS/GEE/%s.geojson' %(name), 'w') as file:
        geojson.dump(fc, file)

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime
from irrigation_detection import search_files

In [ ]:
#Create geopandas geodataframe from all files 
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in glob('C://Users/USER/Desktop/Master_Irrigation/03_GIS/GEE/*')],ignore_index=True))

In [ ]:
#get dates as datetime.datetime for all filenames
dates = [datetime.strptime(x.split('\\')[1][17:32], '%Y%m%dT%H%M%S') for x in glob('C://Users/USER/Desktop/Master_Irrigation/03_GIS/GEE/*')]
sentinel = [x.split('\\')[1][2] for x in glob('C://Users/USER/Desktop/Master_Irrigation/03_GIS/GEE/*')]

In [ ]:
# add dates to column by multiply dates by field numbers 
gdf['date'] = np.array([[x] * len(gdf['id'].unique()) for x in dates]).flatten()

In [ ]:
# add sentinel satellite description to column by multiply dates by field numbers
gdf['sentinel'] = np.array([[x] * len(gdf['id'].unique()) for x in sentinel]).flatten()

In [ ]:
# save dataframe to disk
gdf.to_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\sentinel1\fields01_sentinel1_grd_median.geojson')

In [ ]:
#create poi for irrigated field at 2020-04-26
from shapely.geometry import Point
poi = Point(8.462525010108948, 49.84003060414562)

In [ ]:
# isolate field within point
field01 = gdf[gdf.contains(poi)]
field01

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
import hvplot
import hvplot.pandas

In [ ]:
field01.hvplot(kind='scatter', x='date.day', y='VV', by='sentinel', xticks=30)

In [ ]:
# Create imagecollection by mapping reduceToImage function over FeatureCollection
s1_grd_rr_IC = ee.ImageCollection(s1_grd_rr.map(lambda x: ee.FeatureCollection(x).filter(ee.Filter.notNull(['VV'])).reduceToImage(properties=['VV'], reducer=ee.Reducer.first()).unmask(0).reproject('epsg:4326',None,10).clip(geojsonFc.geometry())))

In [ ]:
url = s1_grd_rr_IC.first().select('first').getThumbURL({'min': -20, 'max': 1})
disp.Image(url=url, width=800)

In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-04-20'), ee.Date('2020-04-30')) 
                       .first() 
                       .clip(aoi))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-04-20'), ee.Date('2020-04-30')) 
                       .first() 
                       .clip(aoi))

In [ ]:
s1_grd_rr_IC.first().getInfo()

In [ ]:
#location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
#rgb = ee.Image.rgb(ffa_db.select('VV'),
#                   ffa_db.select('VH'),
#                   ffa_db.select('VV').divide(ffa_db.select('VH')))
test = s1_grd_rr_IC.first()
# Create the map object.
m = folium.Map(location=['49.983461596534646','8.471317291259766'], zoom_start=15)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(test, {'min': [-20], 'max': [1]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.471317291259766,
              49.983461596534646
            ],
            [
              8.481616973876953,
              49.94867940605311
            ],
            [
              8.498611450195312,
              49.946249234682774
            ],
            [
              8.529338836669922,
              49.951882626433914
            ],
            [
              8.54710578918457,
              49.966239272192034
            ],
            [
              8.471317291259766,
              49.983461596534646
            ]
          ]
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi_sub = ee.Geometry.Polygon(coords)


In [ ]:
hist = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.fixedHistogram(0, 0.5, 500),aoi_sub).get('VV').getInfo()
mean = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.mean(), aoi_sub).get('VV').getInfo()
variance = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.variance(), aoi_sub).get('VV').getInfo()


In [ ]:
a = np.array(hist)
x = a[:, 0]                 # array of bucket edge positions
y = a[:, 1]/np.sum(a[:, 1]) # normalized array of bucket contents
plt.grid()
plt.plot(x, y, '.')
plt.show()


In [ ]:
alpha = 5
beta = mean/alpha
plt.grid()
plt.plot(x, y, '.', label='data')
plt.plot(x, gamma.pdf(x, alpha, 0, beta)/1000, '-r', label='gamma')
plt.legend()
plt.show()


In [ ]:
def X(n):
    return np.sum(np.cos(4*np.pi*(np.random.rand(n)-0.5)))/np.sqrt(n/2)

n= 10000
Xs = [X(n) for i in range(10000)]
y, x = np.histogram(Xs, 100, range=[-5,5])
plt.plot(x[:-1], y/1000, 'b.', label='simulated data')
plt.plot(x, norm.pdf(x), '-r', label='normal distribution')
plt.grid()
plt.legend()
plt.show()


In [ ]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                .filterBounds(aoi)
                .filterDate(ee.Date('2020-08-01'),ee.Date('2020-08-31'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                .filter(ee.Filter.eq('relativeOrbitNumber_start', 15))
                .sort('system:time_start'))


In [ ]:
import time
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]


In [ ]:
im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(2)).select('VV').clip(aoi_sub)
im2 = ee.Image(im_list.get(5)).select('VV').clip(aoi_sub)

In [ ]:
ratio = im1.divide(im2)
url = ratio.getThumbURL({'min': 0, 'max': 10})
disp.Image(url=url, width=800)

In [ ]:
hist = ratio.reduceRegion(ee.Reducer.fixedHistogram(0, 5, 500), aoi_sub).get('VV').getInfo()
mean = ratio.reduceRegion(ee.Reducer.mean(), aoi_sub).get('VV').getInfo()
variance = ratio.reduceRegion(ee.Reducer.variance(), aoi_sub).get('VV').getInfo()


In [ ]:
im1 = ee.Image(im_list.get(2)).select('VV').clip(aoi)
im2 = ee.Image(im_list.get(5)).select('VV').clip(aoi)
ratio = im1.divide(im2)

location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=12)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_child(folium.LayerControl())

display(mp)


In [ ]:
# Decision threshold alpha/2:
dt = f.ppf(0.0005, 2*m, 2*m)

# LRT statistics.
q1 = im1.divide(im2)
q2 = im2.divide(im1)

# Change map with 0 = no change, 1 = decrease, 2 = increase in intensity.
c_map = im1.multiply(0).where(q2.lt(dt), 1)
c_map = c_map.where(q1.lt(dt), 2)

# Mask no-change pixels.
c_map = c_map.updateMask(c_map.gt(0))

# Display map with red for increase and blue for decrease in intensity.
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(
    location=location, tiles='Stamen Toner',
    zoom_start=13)
folium.TileLayer('OpenStreetMap').add_to(mp)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_ee_layer(c_map,
                {'min': 0, 'max': 2, 'palette': ['black', 'blue', 'red']},
                'Change Map')
mp.add_child(folium.LayerControl())

display(mp)
